<a href="https://colab.research.google.com/github/antonioalbanese/Time-Series-Anomaly-Detection-An-experimental-survey/blob/main/Experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install torchinfo --quiet

!pip install wandb --quiet

import os
os.environ["WANDB_API_KEY"] = "571001027d1947e02fe01237fe2d0bf5bbf8071b"

     |████████████████████████████████| 1.9 MB 14.5 MB/s 
     |████████████████████████████████| 168 kB 59.0 MB/s 
     |████████████████████████████████| 182 kB 51.7 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
     |████████████████████████████████| 168 kB 66.2 MB/s 
     |████████████████████████████████| 166 kB 71.5 MB/s 
     |████████████████████████████████| 166 kB 59.4 MB/s 
     |████████████████████████████████| 162 kB 70.0 MB/s 
     |████████████████████████████████| 162 kB 68.7 MB/s 
     |████████████████████████████████| 158 kB 72.3 MB/s 
     |████████████████████████████████| 157 kB 68.1 MB/s 
     |████████████████████████████████| 157 kB 64.2 MB/s 
     |████████████████████████████████| 157 kB 67.4 MB/s 
     |████████████████████████████████| 157 kB 66.4 MB/s 
     |████████████████████████████████| 157 kB 65.8 MB/s 
     |████████████████████████████████| 157 kB 68.5 MB/s 
     |████████████████████████████████| 157 kB 72.9 MB/s 
     |██████████

In [3]:
import zipfile
import numpy as np


!pip install torchinfo --quiet

!git clone https://github.com/antonioalbanese/Time-Series-Anomaly-Detection-An-experimental-survey/

%cd Time-Series-Anomaly-Detection-An-experimental-survey

%mkdir data

%mkdir ./data/SMD
!python USAD/gdrivedl.py https://drive.google.com/file/d/18JNYBsaX7tu0Qfgo92nCBCklv471L1xB data/SMD
with zipfile.ZipFile("./data/SMD/SMD.zip", 'r') as zip_ref:
    zip_ref.extractall("./data/")

%mkdir ./data/MSL
!python USAD/gdrivedl.py https://drive.google.com/uc?id=1ZCLBU_pKTbsPlcj_LwxZE3IRy6mrlys3 data/MSL
with zipfile.ZipFile("./data/MSL/MSL.zip", 'r') as zip_ref:
    zip_ref.extractall("./data/")

!mkdir data/SWAT
#normal period
!python USAD/gdrivedl.py https://drive.google.com/open?id=1rVJ5ry5GG-ZZi5yI4x9lICB8VhErXwCw data/SWAT
#anomalies
!python USAD/gdrivedl.py https://drive.google.com/open?id=1iDYc0OEmidN712fquOBRFjln90SbpaE7 data/SWAT

Cloning into 'Time-Series-Anomaly-Detection-An-experimental-survey'...
remote: Enumerating objects: 1222, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 1222 (delta 18), reused 0 (delta 0), pack-reused 1190
Receiving objects: 100% (1222/1222), 3.07 MiB | 12.15 MiB/s, done.
Resolving deltas: 100% (757/757), done.
/content/Time-Series-Anomaly-Detection-An-experimental-survey
data/SMD/SMD.zip
[==================================================] 185.82MB/185.82MB
data/MSL/MSL.zip
[==================================================] 0.73MB/0.73MB
data/SWAT/SWaT_Dataset_Normal_v1.csv
[==================================================] 163.77MB/163.77MB
data/SWAT/SWaT_Dataset_Attack_v0.csv
[==================================================] 127.27MB/127.27MB


In [ ]:
### NEW TANOGAN 
from AnomalyDetectionMethodClass import ADMethod

configuration = {
    'DATASET': 'SWAT', #SWAT, MSL, SMD, NAB
    'DATAPATH': "realKnownCause/machine_temperature_system_failure.csv", #"realKnownCause/machine_temperature_system_failure.csv", #Only needed with NAB
    'SEQ_LEN': 100,
    'STEP': 100,
    'HIDDEN_SIZE': 100, #Needed for USAD method
    'LR': 0.001,
    'EPOCHS': 20,
    'VERBOSE': True,
    'LOGGER': False
}
# configuration = configurations[curr_config]
method = ADMethod(name = 'TANOGAN', config = configuration)
train_history = method.train()
_, score = method.test()
rep = method.results(0.6, True)

In [11]:
### NEW TRANSFORMER 
from AnomalyDetectionMethodClass import ADMethod

configuration = {
    'DATASET': 'SWAT', #SWAT, MSL, SMD, NAB
    'DATAPATH': "realKnownCause/machine_temperature_system_failure.csv", #"realKnownCause/machine_temperature_system_failure.csv", #Only needed with NAB
    'SEQ_LEN': 100,
    'STEP': 100,
    'HIDDEN_SIZE': 100, #Needed for USAD method
    'LR': 0.001,
    'EPOCHS': 20,
    'K': 3, 
    'VERBOSE': True,
    'LOGGER': False
}
# configuration = configurations[curr_config]
method = ADMethod(name = 'TRANSFORMER', config = configuration)
train_history = method.train()
_, score = method.test()
rep = method.results(0.6, True)

Initializing...


/content/Time-Series-Anomaly-Detection-An-experimental-survey/datafactory.py:25: DtypeWarning:

Columns (26) have mixed types.Specify dtype option on import or set low_memory=False.

/content/Time-Series-Anomaly-Detection-An-experimental-survey/datafactory.py:25: DtypeWarning:

Columns (1,9,28,46) have mixed types.Specify dtype option on import or set low_memory=False.

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning:

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



Data preprocessing and method configuration finished in 27.2076997756958 sec.
Model summary: 
Layer (type:depth-idx)                             Param #
AnomalyTransformer                                 --
├─DataEmbedding: 1-1                               --
│    └─TokenEmbedding: 2-1                         --
│    │    └─Conv1d: 3-1                            78,336
│    └─PositionalEmbedding: 2-2                    --
│    └─Dropout: 2-3                                --
├─Encoder: 1-2                                     --
│    └─ModuleList: 2-4                             --
│    │    └─EncoderLayer: 3-2                      1,583,112
│    │    └─EncoderLayer: 3-3                      1,583,112
│    │    └─EncoderLayer: 3-4                      1,583,112
│    └─LayerNorm: 2-5                              1,024
├─Linear: 1-3                                      26,163
Total params: 4,854,859
Trainable params: 4,854,859
Non-trainable params: 0
Training...
Epoch 1/20: train_loss_1:

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: DtypeWarning:

Columns (1,9,28,46) have mixed types.Specify dtype option on import or set low_memory=False.



              precision    recall  f1-score   support

       False       0.96      0.99      0.97      3917
        True       0.92      0.71      0.80       582

    accuracy                           0.95      4499
   macro avg       0.94      0.85      0.89      4499
weighted avg       0.95      0.95      0.95      4499



In [10]:
rep = method.results(0.55, True)

Computing results... 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DtypeWarning:

Columns (1,9,28,46) have mixed types.Specify dtype option on import or set low_memory=False.



              precision    recall  f1-score   support

       False       0.96      1.00      0.98      3917
        True       0.99      0.69      0.81       582

    accuracy                           0.96      4499
   macro avg       0.97      0.84      0.89      4499
weighted avg       0.96      0.96      0.96      4499



In [6]:
len(method.scores)

449900

In [3]:
import random

random.seed(277)

In [4]:
configurations = []
seq_lens = [10, 20, 10, 300]
for seq_len in seq_lens:
  steps = [1, seq_len]
  for step in steps:
    epochs = [30,50,80]
    lrs= [0.00001, 0.000001, 0.0000001]
    for e, lr in zip(epochs, lrs):
      conf = {
          'DATASET': 'SWAT', #SWAT, MSL, SMD, NAB
          'DATAPATH': None, #"realKnownCause/machine_temperature_system_failure.csv", #Only needed with NAB
          'SEQ_LEN': seq_len,
          'STEP': step,
          'HIDDEN_SIZE': 100, #Needed for USAD method
          'LR': lr,
          'EPOCHS': e, 
          'VERBOSE': False,
          'LOGGER': True
      }
      configurations.append(conf)

In [ ]:
### NEW USAD 
from AnomalyDetectionMethodClass import ADMethod

configuration = {
    'DATASET': 'NAB', #SWAT, MSL, SMD, NAB
    'DATAPATH': "realKnownCause/machine_temperature_system_failure.csv", #"realKnownCause/machine_temperature_system_failure.csv", #Only needed with NAB
    'SEQ_LEN': 100,
    'STEP': 100,
    'HIDDEN_SIZE': 100, #Needed for USAD method
    'LR': 0.0001,
    'EPOCHS': 1, 
    'VERBOSE': True,
    'LOGGER': False
}
# configuration = configurations[curr_config]
method = ADMethod(name = 'USAD', config = configuration)
train_history = method.train()
_, score = method.test()
rep = method.results(0.5, True)

In [ ]:
curr_config = 11 #add one per each run 

### NEW USAD 
from AnomalyDetectionMethodClass import ADMethod

# configuration = {
#     'DATASET': 'SWAT', #SWAT, MSL, SMD, NAB
#     'DATAPATH': None, #"realKnownCause/machine_temperature_system_failure.csv", #Only needed with NAB
#     'SEQ_LEN': 100,
#     'STEP': 100,
#     'HIDDEN_SIZE': 100, #Needed for USAD method
#     'LR': 0.0001,
#     'EPOCHS': 30, 
#     'VERBOSE': True,
#     'LOGGER': True
# }
configuration = configurations[curr_config]
method = ADMethod(name = 'USAD', config = configuration)
train_history = method.train()
_, score = method.test()
for th in np.linspace(0,1,10,endpoint=False):
  rep = method.results(threshold = th, plot = False)

method.close_run()

In [ ]:
### NEW DEEPANT 
from AnomalyDetectionMethodClass import ADMethod

configuration = {
    'DATASET': 'SWAT', #SWAT, MSL, SMD, NAB
    'DATAPATH': None, #"realKnownCause/machine_temperature_system_failure.csv", #Only needed with NAB
    'SEQ_LEN': 100,
    'STEP': 100,
    'HIDDEN_SIZE': None, #Needed for USAD method
    'LR': 0.000001,
    'EPOCHS': 130, #50 for SWAT
    'VERBOSE': True,
    'LOGGER': True
}
method = ADMethod(name = 'DEEPANT', config = configuration)
train_losses = method.train()
predictions, score = method.test()
for th in np.linspace(0,1,10,endpoint=False):
  rep = method.results(threshold = th, plot = False)
method.close_run()

Initializing...


/content/Time-Series-Anomaly-Detection-An-experimental-survey/datafactory.py:25: DtypeWarning:

Columns (26) have mixed types.Specify dtype option on import or set low_memory=False.

/content/Time-Series-Anomaly-Detection-An-experimental-survey/datafactory.py:25: DtypeWarning:

Columns (1,9,28,46) have mixed types.Specify dtype option on import or set low_memory=False.



Data preprocessing and method configuration finished in 26.244796991348267 sec.
Model summary: 
Layer (type:depth-idx)                   Param #
DeepAnt                                  --
├─Sequential: 1-1                        --
│    └─Conv1d: 2-1                       4,928
│    └─ReLU: 2-2                         --
│    └─MaxPool1d: 2-3                    --
├─Sequential: 1-2                        --
│    └─Conv1d: 2-4                       3,104
│    └─ReLU: 2-5                         --
│    └─MaxPool1d: 2-6                    --
├─Flatten: 1-3                           --
├─Sequential: 1-4                        --
│    └─Linear: 2-7                       29,480
│    └─ReLU: 2-8                         --
│    └─Dropout: 2-9                      --
├─Linear: 1-5                            2,091
Total params: 39,603
Trainable params: 39,603
Non-trainable params: 0
Training...


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning:

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



Epoch 1/130: train_loss:0.52825
Epoch 2/130: train_loss:0.52816
Epoch 3/130: train_loss:0.52785
Epoch 4/130: train_loss:0.52743
Epoch 5/130: train_loss:0.52684
Epoch 6/130: train_loss:0.52612
Epoch 7/130: train_loss:0.52518
Epoch 8/130: train_loss:0.52411
Epoch 9/130: train_loss:0.52295
Epoch 10/130: train_loss:0.52168
Epoch 11/130: train_loss:0.52040
Epoch 12/130: train_loss:0.51902
Epoch 13/130: train_loss:0.51761
Epoch 14/130: train_loss:0.51616
Epoch 15/130: train_loss:0.51467
Epoch 16/130: train_loss:0.51312
Epoch 17/130: train_loss:0.51160
Epoch 18/130: train_loss:0.50998
Epoch 19/130: train_loss:0.50834
Epoch 20/130: train_loss:0.50675
Epoch 21/130: train_loss:0.50503
Epoch 22/130: train_loss:0.50329
Epoch 23/130: train_loss:0.50156
Epoch 24/130: train_loss:0.49976
Epoch 25/130: train_loss:0.49805
Epoch 26/130: train_loss:0.49631
Epoch 27/130: train_loss:0.49457
Epoch 28/130: train_loss:0.49277
Epoch 29/130: train_loss:0.49103
Epoch 30/130: train_loss:0.48931
Epoch 31/130: train

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DtypeWarning:

Columns (1,9,28,46) have mixed types.Specify dtype option on import or set low_memory=False.



              precision    recall  f1-score   support

       False       0.95      1.00      0.97      3917
        True       1.00      0.62      0.76       582

    accuracy                           0.95      4499
   macro avg       0.97      0.81      0.87      4499
weighted avg       0.95      0.95      0.95      4499

Computing results... 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DtypeWarning:

Columns (1,9,28,46) have mixed types.Specify dtype option on import or set low_memory=False.



              precision    recall  f1-score   support

       False       0.95      1.00      0.97      3917
        True       1.00      0.62      0.76       582

    accuracy                           0.95      4499
   macro avg       0.97      0.81      0.87      4499
weighted avg       0.95      0.95      0.95      4499

Run closed.
Best accuracy is 0.95 with threshold 0.50
Best Avg f1-score is 0.88 with threshold 0.50
Best True f1-score is 0.79 with threshold 0.40


F1_Avg,▁▄▆▇██████
F1_True,▁▂▄▆██████
Precision_Avg,▁▂▂▄▇█████
Precision_True,▁▂▂▄▇█████
Recall_Avg,▁▅▇██▇▇▇▇▇
Recall_True,█▇▆▅▃▂▁▁▁▁
accuracy,▁▄▆▇██████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,███████▇▇▇▇▇▇▇▇▆▆▆▅▅▅▄▄▄▃▃▂▂▂▂▂▂▂▁▂▂▁▁▁▁
threshold,▁▂▃▃▄▅▆▆▇█
F1_Avg,0.86839


In [ ]:
method.ground.shape, method.anomalies.shape

((4499,), (2699,))

In [ ]:
### USAD
from AnomalyDetectionMethodClass import ADMethod_back
settings_={
    'EPOCHS':20,
    'BATCH_SIZE': 256,
    'SEQ_LEN': 12,
    'HIDDEN_SIZE': 100,
    'ALPHA': 0.5,
    'BETA': 0.5,
    'CONFIDENCE': 0.82,
    'DATASET': 'SWAT'}
data_path = ''
method_ = ADMethod_back('USAD',settings_,'')

method_.prepare_pipeline()
#h = method.train()
#scores, report = method.test()

/content/Time-Series-Anomaly-Detection-An-experimental-survey/USAD/USADSolver.py:22: DtypeWarning: Columns (26) have mixed types.Specify dtype option on import or set low_memory=False.
  self.prepare_data()
/content/Time-Series-Anomaly-Detection-An-experimental-survey/USAD/USADSolver.py:22: DtypeWarning: Columns (1,9,28,46) have mixed types.Specify dtype option on import or set low_memory=False.
  self.prepare_data()


(494988, 12, 51)


In [ ]:
for [el] in method_.solver.train_loader:
  print(el.size())
  break
  

torch.Size([256, 612])


In [ ]:
for el in method.train_dl:
  print(el.size())
  break



torch.Size([256, 612])


In [ ]:
for el in method.solver.train_loader:
  print(el[0].shape)
  print(len(el))
  break

torch.Size([7919, 612])
1


In [ ]:
###  TRANSFORMER
from AnomalyDetectionMethodClass import ADMethod
data_path = './dataset/MSL'
settings = {
    'lr':1e-4,
    'num_epochs':3,
    'k':3,
    'win_size':100,
    'input_c':55,
    'output_c':55,
    'batch_size':256,
    'pretrained_model':None,
    'dataset':'MSL',
    'mode':'train', # choices=['train', 'test']
    'data_path':'./dataset/MSL',
    'model_save_path':'checkpoints',
    'anormly_ratio':1
}
method = ADMethod('transformer',settings, data_path)

method.prepare_pipeline()
method.train()
scores, report = method.test()

In [ ]:
### DEEPANT
from AnomalyDetectionMethodClass import ADMethod
# data_path = './NAB/realKnownCause/cpu_utilization_asg_misconfiguration.csv'
data_path = './dataset/MSL'


LR_LIST = [0.001, 0.0001, 0.00001, 0.000001]
SEQ_LIST = [100, 30, 100, 300]
for lr in [0.0001]:#LR_LIST:
  for seq in [100]: #SEQ_LIST:
    # wandb.init(project="experimental-survey-AD", entity="michiamoantonio", group='DeepAnt')
    # wandb.log({'lr': lr})
    # wandb.log({'seq_len': seq})
    settings = {
      'SEQ_LEN': seq,
      'out_dim': 1,
      'EPOCHS': 1,
      'LR': lr,
      'CONFIDENCE': 0.50,
      'TH_LIST': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
      'TH_SEARCH': True,
      'VERBOSE': True,
      'DATASET': 'SWAT'
    }
    # wandb.config = settings
    method = ADMethod('DeepAnt',settings, data_path)

    method.prepare_pipeline()
    l = method.train()
    scores, report = method.test()

    print(f"lr: {lr}, seq_len: {seq}")
    for i,el in enumerate(report): 
      print(f"#### th: {settings['TH_LIST'][i]}, f1-score: {el['macro avg']['f1-score']}")
      # wandb.log({'th': settings['TH_LIST'][i], 'f1-score': el['macro avg']['f1-score']})
    # wandb.finish()



LOAD TRAIN DATA


/content/Time-Series-Anomaly-Detection-An-experimental-survey/DeepAnt/DeepAntSolver.py:22: DtypeWarning: Columns (26) have mixed types.Specify dtype option on import or set low_memory=False.
  self.prepare_data(data_path)


SCALER
SCALED
window
WINDOWED


In [ ]:
!pip install torchinfo
from torchinfo import summary
summary(method.solver.model)